In [ ]:
%matplotlib widget

In [ ]:
from fimpylab import LightsheetExperiment

In [ ]:
from pathlib import Path
import numpy as np
from matplotlib import  pyplot as plt
import flammkuchen as fl
import pandas as pd

from fimpylab import autoload_experiment

import pandas as pd

import seaborn as sns
from tqdm import tqdm
sns.set(style="ticks", palette="deep")
cols = sns.color_palette()

plt.rcParams['figure.constrained_layout.use'] = True
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Libertinus Sans']

# %autoreload
from lotr.utils import zscore
from lotr.data_loading import preprocess_traces
from lotr.pca import pca_and_phase, get_fictive_trajectory, fictive_trajectory_and_fit

In [ ]:
plt.close("all")
path = Path("/Users/luigipetrucco/Desktop/source_data_batch2/210805_f0_clol")
traces = fl.load(path / "filtered_traces.h5", "/detr")

In [ ]:
df = fl.load(path / "bouts_df.h5")# exp.get_bout_properties()

sel = (df["t_start"] > 10) & (df["t_start"] < 2020)
plt.figure(figsize=(5, 2))
plt.hist(df["bias"], 10, density=True, zorder=-100)
plt.scatter(df["bias"], df["peak_vig"], 100, c=cols[1])

start = 0 #4800
end = 7000
traces = traces[start:end, :]
idx_l = (df.loc[(df["bias"] > 0.2) & (df["t_start"] > 10), 
                "t_start"].values*5 - start).astype(np.int)
idx_r = (df.loc[(df["bias"] < -0.2) & (df["t_start"] > 10), 
                "t_start"].values*5 - start).astype(np.int)
idx_l = idx_l[(idx_l > 0) & (idx_l < traces.shape[0])]
idx_r = idx_r[(idx_r > 0) & (idx_r < traces.shape[0])]
bouts_traces = np.zeros(traces.shape[0])
bouts_traces[idx_l] = 1
bouts_traces[idx_r] = -1

In [ ]:
reg_df = fl.load(path / "motor_regressors.h5")
cc_motor = reg_df["all_bias_abs"].values
cc_motor_integr = reg_df["all_bias_abs_dfdt"].values

In [ ]:
t_lims = (1000, 5100)
t_slice = slice(*t_lims)

corr_mat = np.corrcoef(traces[t_slice, :].T)
n_pairs = 5
selected = []
incl_cc = []
for n in range(n_pairs):
    pair = np.unravel_index(np.nanargmin(corr_mat), corr_mat.shape)
    selected.extend(pair)
    incl_cc.append(corr_mat[pair[0], pair[1]])
    for p in pair:
        corr_mat[p, :] = np.nan
        corr_mat[:, p] = np.nan
np.array((selected))

corr_mat = np.corrcoef(traces[:, :].T)

f = plt.figure(figsize=(3, 3))
plt.scatter(cc_motor, cc_motor_integr, s=10, label="excluded")
#plt.scatter(cc_motor[selected], cc_motor_integr[selected])
x = np.arange(-0.2, cc_motor.max(), 0.05)

c = 0.5
o = 0.05
plt.plot(x, x*c +o, c=(0.3,)*3)
selected = np.argwhere(np.min(corr_mat, 0) < -0.6)[:, 0]
# selected = fl.load(path / "selected_cc.h5")
# selected = np.argwhere((cc_motor_integr > 0.1) & (np.abs(cc_motor) < 0.3))[:, 0]
#selected = np.argwhere((cc_motor_integr > cc_motor*c + o) & (np.abs(cc_motor) < 0.4))[:, 0]
# selected = np.argwhere((cc_motor_integr > cc_motor*c + o))[:, 0]
# selected = np.argwhere((np.abs(cc_motor) < 0.1))[:, 0]
# selected = np.argwhere((np.min(corr_mat, 0) < -0.5)  & (np.abs(cc_motor) < 0.4))[:, 0]
selected = np.argwhere(((cc_motor_integr > cc_motor*c + o) & (np.abs(cc_motor) < 0.8) & \
                        (np.abs(cc_motor_integr)> 0.05)) | \
                      (np.min(corr_mat, 0) < -0.7))[:, 0]
#selected = [ 344,  518,  611,  775,  780,  905,  971,  976,  979,  980,  986,
#       1067, 1078, 1100, 1112, 1188, 1234]
#selected = np.concatenate([selected, np.argmin(corr_mat[selected, :], axis=1)])
plt.ylim(-0.3, 0.8)
plt.xlim(-0.6, 1)
#selected = selected[(np.abs(cc_motor[selected]) < 0.2)]
plt.scatter(cc_motor[selected], cc_motor_integr[selected], s=10, label="included")
# pcaed, phase = pca_and_phase(traces[2000:8000, selected], traces[:, selected])
plt.legend(frameon=False)
plt.xlabel("cc. traces - motor regressor")
plt.ylabel("cc. d(traces)/dt - regressor")
sns.despine()
# f.savefig("/Users/luigipetrucco/Desktop/regbased_incl.pdf")

In [ ]:
pcaed_t, phase_t = pca_and_phase(traces[t_slice, selected].T, traces[t_slice, selected].T)

plt.figure(figsize=(7, 3))
thr = 35
sel = (pcaed_t[:, 0]**2+pcaed_t[:, 1]**2)**(1/2) > thr
plt.scatter(pcaed_t[:, 0], pcaed_t[:, 1], c=sel)
plt.axis("equal")

#selected = selected[sel]
# pcaed, phase = pca_and_phase(traces[t_slice, selected], traces[:, selected])
#pcaed_spont, phase_spont = pca_and_phase(traces[t_slice, selected], traces[t_slice, selected])
pcaed, phase = pca_and_phase(traces[t_slice, selected], traces[:, selected])

In [ ]:
plt.figure(figsize=(7, 2.5))
plt.plot(traces[:, selected] + 4)
plt.plot(mot_reg_base, c="k")
print(len(selected))
plt.show()

In [ ]:
# plt.close("all")
mot_t_slice = slice(5100, 10200)
plt.figure(figsize=(7., 3.))
plt.subplot(1,2,1)
phase = np.angle((pcaed[:, 0]) + 1j * (pcaed[:, 1]))
plt.plot(pcaed[t_slice, 0], pcaed[t_slice, 1], 
         c=(0.6,)*3, lw=0.5, zorder=-100) # , c=phase, cmap="twilight", lw=3)
cb = plt.scatter(pcaed[t_slice, 0], pcaed[t_slice, 1], 
                 c=phase[t_slice], lw=0.5, s=5, cmap="twilight",) 
plt.xlim(-15, 15)
plt.ylim(-15, 15)

plt.subplot(1,2,2)
phase = np.angle((pcaed[:, 0]) + 1j * (pcaed[:, 1]))
plt.plot(pcaed[mot_t_slice, 0], pcaed[mot_t_slice, 1], 
         c=(0.6,)*3, lw=0.5, zorder=-100) # , c=phase, cmap="twilight", lw=3)
cb = plt.scatter(pcaed[mot_t_slice, 0], pcaed[mot_t_slice, 1], 
                 c=phase[mot_t_slice], lw=0.5, s=5, cmap="twilight",) 
plt.xlim(-15, 15)
plt.ylim(-15, 15)
# cbar = add_cbar((0.9, 0.83, 0.02, 0.12), cb, label="phase", ticks=[-np.pi/1.1, np.pi/1.1], 
#         ticklabels=["-π", "π"])
sns.despine()

In [ ]:
plt.figure()
plt.imshow

In [ ]:
unwrapped_phase = np.unwrap(phase)
traj, params = fictive_trajectory_and_fit(unwrapped_phase[t_slice], df, min_bias=0.)
print(params)

In [ ]:
np.corrcoef(traj[:5100], unwrapped_phase[:5100])

In [ ]:
plt.figure(figsize=(7, 3))
plt.scatter(np.arange(len(traj[:5100])), unwrapped_phase[:5100]*params[1] + params[0], 
            c=phase[:5100], cmap="twilight", s=1)
plt.plot(traj[:5100], c=cols[1])

In [ ]:
fl.save(path / "selected.h5", selected)

In [ ]:
from sklearn.decomposition import PCA
from circle_fit import hyper_fit
comp0, comp1 = 0, 1

traces_fit = traces[2000:8000, selected].T
traces_transform = traces_fit
if traces_transform is None:
    traces_transform = traces_fit

# Compute PCA and transform traces:
pca = PCA(n_components=5).fit(traces_fit)
pcaed_t = pca.transform(traces_transform)

# Fit circle:
hf_c = hyper_fit(pcaed[:, [comp0, comp1]])

# Compute phase, after subtracting center of the circle
phase_t = np.angle((pcaed_t[:, 0] - hf_c[0]) + 1j * (pcaed_t[:, 1] - hf_c[1]))

plt.figure(figsize=(7, 3))
plt.scatter(pcaed_t[:, 0], pcaed_t[:, 1], c=phase_t, cmap="twilight")
plt.axis("equal")

In [ ]:
# plt.close("all")
plot_t_s = 10
plot_t_pts = int(plot_t_s * fn)

rot_wnd_s = 1
rot_wnd_pts = int(rot_wnd_s * fn)

f, axs = plt.subplots(1, 3, figsize=(8, 3), sharex=True, sharey=True)
# [ax.set_xlabel("PC 1") for ax in axs]
# axs[0].set_ylabel("PC 2")

cbars = []
for x, ax, idx_list, cmap, title in zip(np.arange(3), 
                                     axs, 
                                     [idx_l, idx_r, random_trig], 
                                     ["Blues", "Reds", "gray_r"],
                                     ["Left bouts", "Right bouts", "Shuffle"]):   
    for l in idx_list[:]:
        try:
            crop_seg = pcaed[l:l+plot_t_pts, :2]
            crop_seg = crop_seg - crop_seg[0, :]
            th = np.arctan2(crop_seg[rot_wnd_pts, 0] - crop_seg[0, 0], 
                            crop_seg[rot_wnd_pts, 1] - crop_seg[0, 1])

            rot_mat = np.array([[np.cos(th), np.sin(th)], [-np.sin(th), np.cos(th)]]).T 
            crop_seg = (rot_mat @ crop_seg[:, :].T).T
            ax.plot(crop_seg[::3, 0], crop_seg[::3, 1], c=(0.4,)*3, lw=0.3, zorder=-100)
            ax.scatter(crop_seg[::3, 0], crop_seg[::3, 1], c=np.arange(len(crop_seg[::3, 1])), cmap=cmap, s=1)
            #cp = color_plot(crop_seg[::3, 0], crop_seg[::3, 1], ax=ax,
             #          cmap=cmap, lw=1)
        except IndexError:
            pass
    cbars.append(add_cbar((0.93, 0.8 + 0.026*x, 0.06, 0.023), cp, label="time (s)" if x==2 else "", 
                          ticks=[], orientation="horizontal"))
    ax.set_title(title)
    # elif x == 1:
    # elif x == 2:
    ax.axvline(0, lw=0.5, c=(0.4,)*3)
    ax.axhline(0, lw=0.5, c=(0.4,)*3)

plt.tight_layout()
sns.despine()

cbars[0].set_ticks([0, 0.95])
cbars[0].set_ticklabels([0, plot_t_s])

In [ ]:
plt.close("all")
plot_t_s = 8
plot_t_pts = int(plot_t_s * fn)

f, axs = plt.subplots(1, 3, figsize=(8, 3), sharex=True, sharey=True)
[ax.set_xlabel("PC 1") for ax in axs]
axs[0].set_ylabel("PC 2")

cbars = []
for x, ax, idx_list, cmap, title in zip(np.arange(3), 
                                     axs, 
                                     [idx_l, idx_r, random_trig], 
                                     ["Blues", "Reds", "gray_r"],
                                     ["Left bouts", "Right bouts", "Shuffle"]):   
    for l in idx_list:
        try:
            crop_seg = pcaed[l:l+plot_t_pts, :2]
            # crop_seg = crop_seg - crop_seg[0, :]
            #cp = color_plot(crop_seg[::3, 0], crop_seg[::3, 1], ax=ax,
            #           cmap=cmap, lw=1)
            ax.plot(crop_seg[::3, 0], crop_seg[::3, 1], c=(0.4,)*3, lw=0.3, zorder=-100)
            ax.scatter(crop_seg[::3, 0], crop_seg[::3, 1], c=np.arange(len(crop_seg[::3, 1])), cmap=cmap, s=1)
            
        except (IndexError, ValueError):
            pass
    cbars.append(add_cbar((0.92, 0.8 + 0.026*x, 0.06, 0.023), cp, label="time (s)" if x==2 else "", 
                          ticks=[], orientation="horizontal"))
    ax.set_title(title)
    # elif x == 1:
    # elif x == 2:
        
plt.tight_layout()
sns.despine()

cbars[0].set_ticks([0, 0.8])
cbars[0].set_ticklabels([0, plot_t_s])

In [ ]:
path

In [ ]:
fl.save(path / "selected_new.h5", selected)

In [ ]:
master_path = Path("/Volumes/Shared/experiments/E0040_motions_cardinal_old/v15_playback")
all_list = list(master_path.glob("[0-9]*_f[0-9]*"))
all_valid = list(master_path.glob("[0-9]*_f[0-9]*/selected.h5"))
print(f"{len(all_valid)}/{len(all_list)}")

In [ ]:
plt.close("all")
plot_t_s = 8
plot_t_pts = int(plot_t_s * fn)

f, axs = plt.subplots(1, 3, figsize=(8, 3), sharex=True, sharey=True)
[ax.set_xlabel("PC 1") for ax in axs]
axs[0].set_ylabel("PC 2")

cbars = []
for x, ax, idx_list, cmap, title in zip(np.arange(3), 
                                     axs, 
                                     [idx_l, idx_r, random_trig], 
                                     ["Blues", "Reds", "gray_r"],
                                     ["Left bouts", "Right bouts", "Shuffle"]):   
    for l in idx_list:
        try:
            crop_seg = pcaed[l:l+plot_t_pts, :2]
            # crop_seg = crop_seg - crop_seg[0, :]
            cp = color_plot(crop_seg[:, 0], crop_seg[:, 1], ax=ax,
                       cmap=cmap, lw=1)
        except IndexError:
            pass
    cbars.append(add_cbar((0.92, 0.8 + 0.026*x, 0.06, 0.023), cp, label="time (s)" if x==2 else "", 
                          ticks=[], orientation="horizontal"))
    ax.set_title(title)
    # elif x == 1:
    # elif x == 2:
        
plt.tight_layout()
sns.despine()

cbars[0].set_ticks([0, 0.8])
cbars[0].set_ticklabels([0, plot_t_s])